# US counties COVID 19 dataset
![Corona Virus](https://highereducationplus.com/wp-content/uploads/2020/03/covid-higher-education-plus.jpg)
 
 
Coronaviruses are large group of viruses that cause illness in humans and animals. Rarely, animal coronaviruses can evolve and infect people and then spread between people such as has been seen with MERS and SARS.

* Although most human coronavirus infections are mild, the epidemics of the severe acute respiratory syndrome coronavirus (SARS-CoV) and Middle East respiratory syndrome coronavirus (MERS-CoV), have caused more than 10,000 cumulative cases in the past two decades, with mortality rates of 10% for SARS-CoV.

* The seven-day average of new cases topped the 14-day average in nine states and Washington, D.C., for Aug. 11, according to a Wall Street Journal analysis of Johns Hopkins University data, suggesting that cases were rising in those areas. A month earlier, the seven-day new case average topped the 14-day new case average in 42 states and D.C.

## Upvote My kernal If you like it

# Map View

In [ ]:
from pandas import read_csv, Grouper, DataFrame, concat
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import statsmodels.tsa.api as smt
import numpy as np
from sklearn.metrics import mean_squared_error
import seaborn as sns
from datetime import datetime
from pandas_profiling import ProfileReport
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing import sequence 
from keras.models import Sequential 
from keras.layers import Dense, Embedding ,Dropout
from keras.layers import LSTM 
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import datetime
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from plotly.offline import init_notebook_mode, iplot
from urllib.request import urlopen
import json

init_notebook_mode(connected=True)  

In [ ]:
ds=pd.read_csv("../input/us-counties-covid-19-dataset/us-counties.csv")
raw=ds.copy()

In [ ]:

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)


fig = px.choropleth_mapbox(ds, geojson=counties, locations='fips', color='cases',
                           color_continuous_scale="Viridis",
                           range_color=(0, 20000),
                           mapbox_style="carto-positron",
                           hover_name ="county",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
ds['date']= pd.to_datetime(ds['date'])
ds.Timestamp=ds["date"]
ds.index = ds.Timestamp 
df = ds.resample('D').sum()

# Confirmed Cases in USA (Jan - Aug) 📅

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index,
    y=df.cases,
    name="Cases in USA"     
))




fig.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

### 📌
* Jan 22th -  1     case
* Aug 14th - 5.32M  cases 

 The count is exponentially growing. 

# Death Cases in USA (Jan - Aug) 📅

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index,
    y=df.deaths,
    name="Cases in USA"     
))




fig.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

### 📌
* Upto Aug 14th-  168.018k deaths

#### Comparing to the Confirmed cases , death cases are looking too low , that is a good one. but Afer 3month it can be worst.

# 📎 Growth Rate 📈

 growth rate calculation can be remarkably simple. Basic growth rates are simply expressed as the difference between two values in time in terms of a percentage of the first value.
 
### 1. Calculating Basic Growth Rates
![](https://www.wikihow.com/images/thumb/d/d9/Calculate-Growth-Rate-Step-2-Version-5.jpg/aid734329-v4-728px-Calculate-Growth-Rate-Step-2-Version-5.jpg)


In [ ]:
case=df.cases
case=list(case)
population=case

### For this dataset Growth Rate has been calculated

In [ ]:
growth_rate=[]

def growth():
    for pop in range(1, len(population)):
        gnumbers = (population[pop] / population[pop-1])/ population[pop-1]
        growth_rate.append(gnumbers)

growth()
def Average(growth_rate): 
    return sum(growth_rate) / len(growth_rate) 
  
average = Average(growth_rate) 

print("Now we have the Average growth factor of USA\n\n Growth factor value :",average)

# 2. After 60days case count by Exponential Growth

![](https://nicoletalgebra.weebly.com/uploads/2/4/7/9/24793402/9316745.jpg?195)

where ,  

 a- Last day count 
 
 b- Growth Factor (we have calculated)
 
 X - Time period
 

In [ ]:
val=df.iloc[-1]
val1=val["cases"]
ave='{0:.40f}'.format(average).rstrip("0") 
ave=float(ave)

result=val1*ave*60

In [ ]:
print("Last day count is",val1,"\n\nGrowth factor is ",ave,"\n\n X is ",60,"\n")
print("After 60days case count wil be \n\n",result)


### So , By the Growth Rate after 60 days case count will be 13.15848M (October 13th)

# 📎 Holt-Winters seasonal method 📏 

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2018/01/eq.png)

### When to Use Holt-Winters
Before you select any prediction method, you need to evaluate the characteristics of your dataset. To determine whether your time series data is a good candidate for Holt-Winters or not, you need to make sure that your data:

Isn’t stochastic. If it is random, then it’s actually a good candidate for Single Exponential Smoothing.
Has a trend.
Has seasonality. In other words, your data has patterns at regular intervals. For example, if you were monitoring traffic data, you would see a spike in the middle of the day and a decrease in activity during the night. In this case, your seasonal period might be one day.

### How Single Exponential Smoothing Works
Before we dive into Holt-Winters or Triple Exponential Smoothing, I’ll explain how Single Exponential Smoothing works. Single Exponential Smoothing (SES) is the simplest exponential smoothing method. Exponential smoothing is just a technique for smoothing time-series data where exponentially decreasing weights are assigned to past observations. It is built upon the Naïve Method. With this method, the forecasted value is equal to the last observed value,

Hard to believe that Naïve would want to be credited with such a simple model, but it turns out it’s quite good at predicting financial data. Also, taking the percent difference between the actual and predicted values can be a good way to uncover seasonality.

In [ ]:
start = datetime.datetime.strptime("2020-08-15", "%Y-%m-%d")
end = datetime.datetime.strptime("2020-10-14", "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

dt=[]

for date in date_generated:
    dt.append(date.strftime("%Y-%m-%d"))
dtd=pd.DataFrame()
dtd["Date"]=dt

In [ ]:
fit1 = ExponentialSmoothing(np.asarray(df['cases']) ,seasonal_periods=7 ,trend='add', seasonal='add',).fit()
pred = fit1.forecast(60)
pred=pred.astype(int)
dtd["Holt"]=pred
dtd.Timestamp=dtd["Date"]
dtd.index = dtd.Timestamp 

### Predicting for the next 60 days cases in USA

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index,
    y=df.cases,
    name="Confirmed Cases"     
))

fig.add_trace(go.Scatter(
    x=dtd.index,
    y=dtd.Holt,
    name="Future Prediction"     
))



fig.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

### 📌
* By prediction on October 13th -  8.88M cases
* based on the growth rate and Holt winter , we can see that it increases more than 8M after 6months 

##### It can be also get down if their is any change in the vaccine or lockdown get much strict means.



### Would Continue...